<a href="https://colab.research.google.com/github/ianzin30/Data-Science-Project/blob/main/exploratory_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análise de Saúde e Estilo de Vida

Este notebook contém análises exploratórias de dados do dataset `Sleep_health_and_lifestyle_dataset.csv`, que inclui informações sobre saúde e estilo de vida, como duração do sono, qualidade do sono, nível de atividade física, e outros.

## Importação de Bibliotecas

Aqui, importamos as bibliotecas necessárias para nossa análise de dados.

```python
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns



```python
# Apenas baixando no colab a biblioteca

In [7]:
!pip install fancyimpute


  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 8.2 MB/s eta 0:00:00
  Created wheel for fancyimpute: filename=fancyimpute-0.7.0-py3-none-any.whl size=29881 sha256=fa3652a36d2a6e77bb1b5ce1667f36c28b99ace8020b834e88ce1406c05267b6
  Stored in directory: /root/.cache/pip/wheels/7b/0c/d3/ee82d1fbdcc0858d96434af108608d01703505d453720c84ed
  Created wheel for knnimpute: filename=knnimpute-0.1.0-py3-none-any.whl size=11330 sha256=cbb96432a0a620c86b2f539eb97c6a8ae7ea3a50f237be7ca7b0d3365f7b14c1
  Stored in directory: /root/.cache/pip/wheels/46/06/a5/45a724630562413c374e29c08732411d496092408b3a7bf754
Successfully built fancyimpute knnimpute


In [8]:
import pandas as pd
import fancyimpute
import matplotlib.pyplot as plt
import seaborn as sns

### Leitura dos Dados
Carregamos nosso dataset e exibimos as primeiras 10 linhas para ter uma visão inicial dos dados disponíveis.

In [9]:
# Lendo o arquivo csv e mostrando as 10 primeiras linhas do dataset

dataframe = pd.read_csv('Sleep_health_and_lifestyle_dataset.csv')
dataframe.head(10)

,Person ID,Gender,Age,Occupation,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Blood Pressure,Heart Rate,Daily Steps,Sleep Disorder
0,1,Male,27,Software Engineer,6.1,6,42.0,6,Overweight,126/83,77,4200,NaN
1,2,Male,28,Doctor,6.2,6,60.0,8,Normal,125/80,75,10000,NaN
2,3,Male,28,Doctor,NaN,6,60.0,8,Normal,125/80,75,10000,NaN
3,4,Male,28,Sales Representative,5.9,4,30.0,8,Obese,140/90,85,3000,Sleep Apnea
4,5,Male,28,Sales Representative,5.9,4,30.0,8,Obese,140/90,85,3000,Sleep Apnea
5,6,Male,28,Software Engineer,5.9,4,30.0,8,Obese,140/90,85,3000,Insomnia
6,7,Male,29,Teacher,6.3,6,40.0,7,Obese,140/90,82,3500,Insomnia
7,8,Male,29,Doctor,7.8,7,75.0,6,Normal,120/80,70,8000,NaN
8,9,Male,29,Doctor,7.8,7,75.0,6,Normal,120/80,70,8000,NaN
9,10,Male,29,Doctor,7.8,7,75.0,6,Normal,120/80,70,8000,NaN


In [10]:
# Contando o número de entradas no dataset
print("Número total de entradas no dataset:", len(dataframe))



Número total de entradas no dataset: 382


In [11]:
# Descrição estatística das variáveis numéricas
dataframe.describe()

,Person ID,Age,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,Heart Rate,Daily Steps
count,382.000000,382.000000,375.000000,382.000000,377.000000,382.000000,382.000000,382.000000
mean,189.109948,42.314136,7.136800,7.319372,59.230769,5.379581,70.157068,6825.916230
std,108.171287,8.680733,0.798113,1.200434,20.934734,1.782063,4.128060,1624.414637
min,1.000000,27.000000,5.800000,4.000000,30.000000,3.000000,65.000000,3000.000000
25%,96.250000,36.000000,6.400000,6.000000,45.000000,4.000000,68.000000,5600.000000
50%,189.500000,43.000000,7.200000,7.000000,60.000000,5.000000,70.000000,7000.000000
75%,283.750000,50.000000,7.800000,8.000000,75.000000,7.000000,72.000000,8000.000000
max,374.000000,59.000000,8.500000,9.000000,90.000000,8.000000,86.000000,10000.000000


In [12]:
dataframe.dtypes

Person ID                    int64
Gender                      object
Age                          int64
Occupation                  object
Sleep Duration             float64
Quality of Sleep             int64
Physical Activity Level    float64
Stress Level                 int64
BMI Category                object
Blood Pressure              object
Heart Rate                   int64
Daily Steps                  int64
Sleep Disorder              object
dtype: object

In [13]:
# trocando dados "object" para "category"
dataframe["Gender"] = dataframe["Gender"].astype("category")
dataframe["Occupation"] = dataframe["Occupation"].astype("category")
dataframe["BMI Category"] = dataframe["BMI Category"].astype("category")
dataframe["Sleep Disorder"] = dataframe["Sleep Disorder"].astype("category")

print(dataframe.dtypes)

Person ID                     int64
Gender                     category
Age                           int64
Occupation                 category
Sleep Duration              float64
Quality of Sleep              int64
Physical Activity Level     float64
Stress Level                  int64
BMI Category               category
Blood Pressure               object
Heart Rate                    int64
Daily Steps                   int64
Sleep Disorder             category
dtype: object


## Conclusões sobre a definição de tipos

Ao revisar os tipos de dados no DataFrame, observamos que os tipos estão bem ajustados para a maioria dos campos, mas há oportunidades para otimizações. Identificadores e métricas quantitativas como "Person ID", "Age", e "Quality of Sleep" estão corretamente como int64. "Sleep Duration" está como float64, adequado para valores que necessitam precisão decimal. No entanto, colunas como "Gender", "Occupation", e "Blood Pressure", classificadas como `object`, poderiam ser convertidas para o tipo `categórico`. A conversão para categórico não só economiza memória, especialmente útil para grandes datasets, como também facilita a realização de análises estatísticas que dependem de agrupamento ou comparação de categorias.

## Tratamento de Valores Ausentes

```python
# Valores ausentes podem distorcer análises e afetar a performance de modelos preditivos. Aqui, investigamos e tratamos esses valores, assegurando a integridade de nosso conjunto de dados.


In [ ]:
predict_dataframe = dataframe.copy(deep=True)

predict_dataframe["Blood Pressure"] = predict_dataframe["Blood Pressure"].astype("category")


predict_dataframe["Gender"] = predict_dataframe["Gender"].cat.codes
predict_dataframe["Occupation"] = predict_dataframe["Occupation"].cat.codes
predict_dataframe["BMI Category"] = predict_dataframe["BMI Category"].cat.codes
predict_dataframe["Sleep Disorder"] = predict_dataframe["Sleep Disorder"].cat.codes
predict_dataframe["Blood Pressure"] = predict_dataframe["Blood Pressure"].cat.codes
predict_dataframe.head()

```python
# Mostrando o número de valores ausentes por coluna
print(dataframe.isnull().sum())

In [ ]:
print(predict_dataframe.isnull().sum())

## Imputação de Valores Ausentes usando K-NN

```python
# Para tratar valores ausentes, utilizamos o método K-Nearest Neighbors (K-NN) que prevê os valores ausentes com base nos vizinhos mais próximos. Este método é particularmente útil para dados que podem ter padrões complexos que modelos mais simples não conseguiriam capturar adequadamente.

from fancyimpute import KNN

In [ ]:
from fancyimpute import KNN

# Aplicando KNN
sleep_knn = KNN(k=3).fit_transform(predict_dataframe)

In [ ]:
# Convertendo o resultado para um DataFrame
sleep_knn = pd.DataFrame(data=sleep_knn[0:, 0:], columns=["Person ID", "Gender", "Age", "Occupation", "Sleep Duration", 'Quality of Sleep', "Physical Activity Level", "Stress Level", "BMI Category", "Blood Pressure", "Heart Rate", "Daily Steps", "Sleep Disorder"])

In [ ]:
# printando as primeiras 10 linhas do dataset
sleep_knn.head(10)

In [ ]:
# Verificando se ainda existem valores nulos
print(sleep_knn.isnull().sum())

#### Atualização de Tipos de Dados
Após a imputação, garantimos que os tipos de dados estejam corretos, especialmente para variáveis categóricas, onde utilizamos codificação numérica para facilitar análises futuras.

In [ ]:
# Atualizando o DataFrame original com os dados imputados
dataframe["Sleep Duration"] = sleep_knn["Sleep Duration"]
dataframe["Physical Activity Level"] = sleep_knn["Physical Activity Level"]

dataframe.head()

### Lidando com Entries/Rows Duplicadas

Identificamos e removemos entradas duplicadas para garantir a integridade dos dados. Isso é crucial para evitar distorções nas análises.

In [ ]:
len(dataframe)

### Verificando as rows duplicadas

In [ ]:
dataframe[dataframe.duplicated()].sort_values("Person ID").head(10)

In [ ]:
# quantidade de linhas duplicadas
len(dataframe[dataframe.duplicated()])

In [ ]:
# removendo linhas duplicadas
dataframe.drop_duplicates(keep='first', inplace=True)

In [ ]:
len(dataframe[dataframe.duplicated()])

In [ ]:
len(dataframe)

## Normalização dos Dados

A normalização é crucial para garantir que variáveis com escalas diferentes não distorçam os resultados analíticos, especialmente em técnicas de análise que são sensíveis a variações de escala, como modelagem preditiva e algoritmos de machine learning.

```python
# Criação de um novo DataFrame para armazenar os dados normalizados
normalized_dataset = dataframe.copy(deep=True)

In [ ]:
normalized_dataset = dataframe.copy(deep=True)
normalized_dataset.head(10)

### Métodos de Normalização

Utilizamos a normalização Z-Score para as variáveis contínuas, que reescala os dados para terem média zero e desvio padrão de um. Isso é feito subtraindo a média e dividindo pelo desvio padrão de cada variável.

In [ ]:
normalized_dataset["Age Norm"] = ((normalized_dataset["Age"] - normalized_dataset["Age"].mean()) / normalized_dataset["Age"].std())
normalized_dataset["Sleep Norm"] = ((normalized_dataset["Sleep Duration"] - normalized_dataset["Sleep Duration"].mean()) / normalized_dataset["Sleep Duration"].std())
normalized_dataset["Quality Norm"] = ((normalized_dataset["Quality of Sleep"] - normalized_dataset["Quality of Sleep"].mean()) / normalized_dataset["Quality of Sleep"].std())
normalized_dataset["Physical Norm"] = ((normalized_dataset["Physical Activity Level"] - normalized_dataset["Physical Activity Level"].mean()) / normalized_dataset["Physical Activity Level"].std())
normalized_dataset["Stress Norm"] = ((normalized_dataset["Stress Level"] - normalized_dataset["Stress Level"].mean()) / normalized_dataset["Stress Level"].std())
normalized_dataset["Heart Norm"] = ((normalized_dataset["Heart Rate"] - normalized_dataset["Heart Rate"].mean()) / normalized_dataset["Heart Rate"].std())
normalized_dataset["Steps Norm"] = ((normalized_dataset["Daily Steps"] - normalized_dataset["Daily Steps"].mean()) / normalized_dataset["Daily Steps"].std())
normalized_dataset.head(10)

In [3]:
normalized_dataset.describe()

NameError: name 'normalized_dataset' is not defined


### Explicação Adicional

- **Por que normalizar?** Algoritmos como K-Means clustering, PCA, regressão linear, e redes neurais são sensíveis à escala das variáveis de entrada. Variáveis com grandes variações podem influenciar desproporcionalmente o modelo, levando a resultados enviesados ou subótimos.
- **Impacto da Normalização**: Além de facilitar a modelagem, a normalização também ajuda na visualização dos dados, tornando mais fácil identificar padrões e outliers em escalas comparáveis.

---


# Discretização de Dados

Discretizar uma variável contínua transforma-a em várias categorias com intervalos definidos. Isso é útil em análises que requerem agrupamento de dados ou quando os algoritmos de machine learning se beneficiam de variáveis categóricas.

```python
# Cópia do dataset para manter o original intacto
discretized_dataset = normalized_dataset.copy(deep=True)



In [ ]:
discretized_dataset = normalized_dataset.copy(deep=True)
discretized_dataset.head(10)

```python
# Discretização da variável 'Sleep Duration' em 10 intervalos

In [ ]:
discretized_dataset["Sleep Discretized"] = pd.cut(discretized_dataset["Sleep Duration"], 10)
discretized_dataset.head(10)

In [ ]:
discretized_dataset["Sleep Discretized"].value_counts()


### Explicação Adicional

- **Por que discretizar?** A discretização ajuda na interpretação dos modelos e na análise, transformando dados contínuos em um formato que imita a maneira como os humanos categorizam naturalmente informações (como curto, médio e longo).
- **Impactos da discretização**: Além de facilitar a interpretação, a discretização pode melhorar a performance de certos algoritmos de classificação, ao reduzir a sensibilidade a pequenas flutuações nos dados contínuos, focando em tendências maiores que são mais robustas a ruídos.

---

## Detecção de Outliers

A presença de outliers pode influenciar negativamente nossas análises e modelos preditivos. Utilizamos box plots para identificar visualmente outliers nas principais variáveis do dataset.

### Visualização de Outliers com Box Plots

Box plots são extremamente úteis para detectar outliers, pois mostram a distribuição dos dados através de quartis e destacam pontos que caem fora do intervalo interquartil estendido.


In [ ]:
discretized_dataset["Age"].plot.box()

In [ ]:
discretized_dataset["Sleep Duration"].plot.box()

In [ ]:
discretized_dataset["Quality of Sleep"].plot.box()

In [ ]:
discretized_dataset["Physical Activity Level"].plot.box()

In [ ]:
discretized_dataset["Stress Level"].plot.box()

In [ ]:
discretized_dataset["Heart Rate"].plot.box()

In [ ]:
discretized_dataset["Daily Steps"].plot.box()

## Remoção e Codificação de Outliers

Após a detecção de outliers, procedemos com a remoção ou tratamento destes para evitar distorções nas análises subsequentes. Em algumas situações, em vez de remover, optamos por codificar os outliers para preservar informações que podem ser úteis.

### Codificação e Tratamento de Outliers

Optamos por codificar variáveis categóricas que contêm outliers, transformando-as de strings para códigos numéricos. Isso simplifica os processos de análise que seguem e ajuda na integração com algoritmos de machine learning que requerem entrada numérica.

```python
# Criando uma cópia do dataset para tratamento de outliers
outlier_dataframe = discretized_dataset.copy(deep=True)

In [ ]:
discretized_dataset

In [ ]:
outlier_dataframe = discretized_dataset.copy(deep=True)

In [ ]:
outlier_dataframe

```python
# Convertendo variáveis categóricas para códigos numéricos

In [ ]:
outlier_dataframe["Blood Pressure"] = outlier_dataframe["Blood Pressure"].astype("category")


outlier_dataframe["Gender"] = outlier_dataframe["Gender"].cat.codes
outlier_dataframe["Occupation"] = outlier_dataframe["Occupation"].cat.codes
outlier_dataframe["BMI Category"] = outlier_dataframe["BMI Category"].cat.codes
outlier_dataframe["Sleep Disorder"] = outlier_dataframe["Sleep Disorder"].cat.codes
outlier_dataframe["Blood Pressure"] = outlier_dataframe["Blood Pressure"].cat.codes
outlier_dataframe.head()

In [ ]:
corr_matrix = outlier_dataframe[['Sleep Duration', 'Quality of Sleep', 'Physical Activity Level', 'Heart Rate', 'BMI Category', 'Blood Pressure', 'Stress Level', 'Daily Steps']].corr()


# Exibindo a matriz de correlação
print(corr_matrix)


## Análise de Correlação do Dataset, útil para decidir técnica de remoção de outliers


#### Resultados da Análise de Correlação
A matriz de correlação gerada fornece informações sobre a força e a direção das relações lineares entre pares de variáveis. Aqui estão as principais conclusões:

1. **Relação Sono e Estresse**:
   - **Duração do Sono e Nível de Estresse**: Correlação negativa forte (-0.811208), indicando que uma maior duração do sono está associada a níveis mais baixos de estresse.
   - **Qualidade do Sono e Nível de Estresse**: Correlação negativa forte (-0.898752), sugerindo que uma melhor qualidade do sono está associada a menores níveis de estresse.

2. **Atividade Física e Saúde**:
   - **Nível de Atividade Física e Passos Diários**: Correlação positiva forte (0.774890), mostrando que um maior nível de atividade física está associado a um maior número de passos diários.
   - **Nível de Atividade Física e Nível de Estresse**: Correlação negativa moderada (-0.670026), indicando que um maior nível de atividade física está associado a níveis mais baixos de estresse.

3. **Qualidade do Sono e Frequência Cardíaca**:
   - Correlação negativa moderada (-0.659865), sugerindo que uma melhor qualidade do sono está associada a uma menor frequência cardíaca.

4. **Índice de Massa Corporal (BMI) e Frequência Cardíaca**:
   - Correlação positiva moderada (0.295558), sugerindo que um maior índice de massa corporal está associado a uma frequência cardíaca mais alta.

#### Conclusões

- **Sono e Estresse**: A duração e a qualidade do sono têm uma forte correlação com os níveis de estresse. Indivíduos que dormem mais e têm uma melhor qualidade de sono tendem a ter níveis de estresse mais baixos.
- **Atividade Física**: O nível de atividade física, medido pelos passos diários, está fortemente correlacionado com a frequência cardíaca e o nível de estresse, sugerindo que a atividade física pode ajudar a reduzir o estresse e controlar a frequência cardíaca.
- **Saúde Geral**: Uma melhor qualidade do sono está associada a uma menor frequência cardíaca, indicando um melhor estado de saúde geral. Além disso, um maior índice de massa corporal está associado a uma frequência cardíaca mais alta, destacando a importância de manter um peso saudável.

#### Próximos Passos

- **Análise Multivariada**: Com base nas correlações encontradas, métodos multivariados como Elliptic Envelope ou Isolation Forests podem ser usados para detectar outliers, considerando as interações entre variáveis.

---

# Porém, não será necessário tratar tais valores

#### Podemos utilizar nosso conhecimento de domínio para afirmar que tais outliers de frequência cardíaca são normais para um humano em repouso, visto que estão em um intervalo aceito de BPMs, além disso, não variam de forma preocupante como pode ser observado nos plots acima, dessa forma, aceitamos que não seria necessário utlizar métodos para outliers que estão de certa forma "corretos".


In [4]:
discretized_dataset["Sleep Duration"].plot.hist()

NameError: name 'discretized_dataset' is not defined

# Análise descritiva

In [ ]:
dataframe.head(10)

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(dataframe['Sleep Duration'], dataframe['Quality of Sleep'], alpha=0.5)
plt.title('Relationship Between Sleep Duration and Quality of Sleep')
plt.xlabel('Sleep Duration')
plt.ylabel('Quality of Sleep')
plt.grid(True)
plt.show()

In [ ]:
import numpy as np

In [ ]:
# Assuming your DataFrame is named df
# First plot the scatter plot
plt.figure(figsize=(10, 6))
plt.scatter(dataframe['Sleep Duration'], dataframe['Quality of Sleep'], alpha=0.5)

# Calculate the coefficients for the line of best fit
slope, intercept = np.polyfit(dataframe['Sleep Duration'], dataframe['Quality of Sleep'], 1)

# Create x values for the line of best fit from the scatter data
x = np.array([dataframe['Sleep Duration'].min(), dataframe['Sleep Duration'].max()])

# Calculate the corresponding y values from the slope and intercept
y = slope * x + intercept

# Plot the line of best fit
plt.plot(x, y, 'r', label='Line of Best Fit')

# Add labels, title, grid, and legend
plt.xlabel('Sleep Duration')
plt.ylabel('Quality of Sleep')
plt.title('Relationship Between Sleep Duration and Quality of Sleep')
plt.grid(True)
plt.legend()

# Show the plot
plt.show()


In [ ]:
# Assuming your DataFrame is named df
# Plotting
plt.figure(figsize=(12, 8))
dataframe.boxplot(column='Daily Steps', by='Occupation', grid=True)
plt.title('Daily Steps by Occupation')
plt.suptitle('')  # Suppress the default title to clean up the plot a little
plt.xlabel('Occupation')
plt.ylabel('Daily Steps')
plt.xticks(rotation=45)  # Rotate the labels on the x-axis for better readability
plt.show()

In [ ]:
dataframe['Daily Steps'].groupby(dataframe['Occupation']).median().plot.bar()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Average Stress Level by Occupation
sns.barplot(x='Occupation', y='Stress Level', data=dataframe)
plt.xticks(rotation=45)
plt.title('Average Stress Level by Occupation')
plt.show()


In [ ]:
# Count of Sleep Disorders by BMI Category
pd.crosstab(dataframe['BMI Category'], dataframe['Sleep Disorder']).plot(kind='bar', stacked=True)
plt.title('Sleep Disorders by BMI Category')
plt.show()



In [ ]:
dataframe['BMI Category'].value_counts()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Scatter plot for Physical Activity Level vs. Heart Rate
sns.scatterplot(x='Physical Activity Level', y='Heart Rate', data=dataframe, hue='BMI Category')
plt.title('Physical Activity vs. Heart Rate by BMI Category')
plt.show()

# Scatter plot for Physical Activity Level vs. Blood Pressure
sns.scatterplot(x='Physical Activity Level', y='Blood Pressure', data=dataframe)
plt.title('Physical Activity vs. Blood Pressure')
plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Gender vs. Stress Level
sns.boxplot(x='Gender', y='Stress Level', data=dataframe)
plt.title('Nível de estresse por gênero')
plt.show()

# Gender vs. Physical Activity Level
sns.barplot(x='Gender', y='Physical Activity Level', data=dataframe)
plt.title('Nível de estresse por gênero')
plt.show()


In [ ]:
# Assuming your DataFrame is named df
# First plot the scatter plot
plt.figure(figsize=(10, 6))
plt.scatter(dataframe['Quality of Sleep'], dataframe['Stress Level'], alpha=0.5)

# Calculate the coefficients for the line of best fit
slope, intercept = np.polyfit(dataframe['Quality of Sleep'], dataframe['Stress Level'], 1)

# Create x values for the line of best fit from the scatter data
x = np.array([ dataframe['Stress Level'].min(), dataframe['Stress Level'].max()])

# Calculate the corresponding y values from the slope and intercept
y = slope * x + intercept

# Plot the line of best fit
plt.plot(x, y, 'r', label='Line of Best Fit')

# Add labels, title, grid, and legend
plt.xlabel('Quality of Sleep')
plt.ylabel('Stress Level')
plt.title('Relationship Between Quality of Sleep and Stress Level')
plt.grid(True)
plt.legend()

# Show the plot
plt.show()


# Teste de hipótese

### Hipótese nula (H0): Não há relação monotônica entre o nível de atividade física e a qualidade do sono.

### Hipótese alternativa (H1): Existe uma relação monotônica (seja crescente ou decrescente) entre o nível de atividade física e a qualidade do sono.


In [ ]:
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.hist(dataframe['Quality of Sleep'], bins=10, color='blue', edgecolor='black')
plt.title('Histogram of Quality of Sleep')

In [ ]:
from scipy.stats import spearmanr, mannwhitneyu

In [ ]:
corr, p_value = spearmanr(dataframe['Physical Activity Level'], dataframe['Quality of Sleep'])
print("Spearman's Correlation Coefficient:", corr)

print("P-value:", p_value)

### 2o teste de hipótese

Hipótese nula(H0): Não tem diferença no nível de estresse entre doutores e engenheiros de software.

Hipótese alternativa(H1): Tem diferença no nível de estresse entre doutores e engenheiros.

In [ ]:
dataframe.head(10)

In [ ]:
dataframe[dataframe['Occupation'] == 'Doctor']['Quality of Sleep'].plot.hist()

In [ ]:
stats.mannwhitneyu(dataframe[dataframe['Occupation'] == 'Doctor']['Quality of Sleep'], dataframe[dataframe['Occupation'] == 'Software Engineer']['Quality of Sleep'])